# Kepler Framework, Emulate v17

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'


In [4]:
tpath_data17     = '../v12/output/emulation/data17_table_v12.h5'
tpath_data17_trk = '../v2_el/output/emulation/data17_table_v2_el.h5'


tpath_data18     = '../v12/output/emulation/data18_table_v12.h5'
tpath_data18_trk = '../v2_el/output/emulation/data18_table_v2_el.h5'

## Setup Chains:

In [6]:
# create my chain
chains = [
    Chain( "HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI"    , L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v12_vloose',l2_column = 'ringer_v2_el_loose'),
    Chain( "HLT_e28_lhtight_nod0_ringer_v17_ivarloose"     , L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v12_tight' ,l2_column = 'ringer_v2_el_loose'),
    Chain( "HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI"    , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v12_medium',l2_column = 'ringer_v2_el_loose'),
    Chain( "HLT_e140_lhloose_nod0_ringer_v17"              , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v12_loose' ,l2_column = 'ringer_v2_el_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v12_vloose')),
             ('L2', 'ringer_v2_el_loose'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v12_tight')),
             ('L2', 'ringer_v2_el_loose'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v12_medium')),
             ('L2', 'ringer_v2_el_loose'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v12_loose')),
             ('L2', 'ringer_v2_el_loose'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [7]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17), load_hdf(tpath_data17_trk)), axis=1)

In [8]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:23<00:00, 20.85s/it]


In [9]:
data17_df.shape

(43311283, 114)

### Validate:

In [10]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v17.root')

2022-03-28 19:23:50,514 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v17.root


In [11]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v17_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v17'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.56s/it]


HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5262  | 29972876/32047563 |
|   L2   | 93.0391  | 29816751/32047563 |
| EFCalo | 91.7671  | 29409117/32047563 |
|  HLT   | 89.8703  | 28801225/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:22<00:00, 16.44s/it]


HLT_e28_lhtight_nod0_ringer_v17_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2246  | 21450792/22765602 |
|   L2   | 94.0800  | 21417876/22765602 |
| EFCalo | 93.1663  | 21209859/22765602 |
|  HLT   | 87.0101  | 19808376/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.3531  | 1331582/1367786 |
|   L2   | 97.1801  | 1329216/1367786 |
| EFCalo | 95.1182  | 1301014/1367786 |
|  HLT   | 91.8576  | 1256416/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.08it/s]

HLT_e140_lhloose_nod0_ringer_v17
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 97.9865  | 59322/60541  |
|   L2   | 97.3109  | 58913/60541  |
| EFCalo | 96.1200  | 58192/60541  |
|  HLT   | 94.2452  | 57057/60541  |
+--------+----------+--------------+


In [12]:
eff_data17_Zee.save()

In [13]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v17.root')

2022-03-28 19:28:03,959 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v17.root


In [14]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v17_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v17'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:21<00:00,  4.35s/it]


HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  2.1368  | 221154/10349992  |
|   L2   |  0.5914  |  61205/10349992  |
| EFCalo |  0.5561  |  57558/10349992  |
|  HLT   |  0.0865  |  8951/10349992   |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.92s/it]


HLT_e28_lhtight_nod0_ringer_v17_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.8239  |  76244/9254300  |
|   L2   |  0.3482  |  32228/9254300  |
| EFCalo |  0.3152  |  29172/9254300  |
|  HLT   |  0.0017  |   154/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.33s/it]


HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.8132  |  63857/7852181  |
|   L2   |  0.2753  |  21617/7852181  |
| EFCalo |  0.2391  |  18776/7852181  |
|  HLT   |  0.0020  |   157/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.45s/it]

HLT_e140_lhloose_nod0_ringer_v17
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  1.9191  |  108121/5634041 |
|   L2   |  0.3570  |  20113/5634041  |
| EFCalo |  0.3180  |  17915/5634041  |
|  HLT   |  0.0216  |   1219/5634041  |
+--------+----------+-----------------+


In [15]:
eff_data17_JF17.save()

In [16]:
del data17_df

## Emulate 2018:

In [17]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18), load_hdf(tpath_data18_trk)), axis=1)

In [18]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:21<00:00, 20.49s/it]


In [19]:
data18_df.shape

(41873973, 114)

In [20]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v17.root')

2022-03-28 19:32:14,778 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v17.root


In [21]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v17_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v17'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.40s/it]


HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5751  | 30524649/32620495 |
|   L2   | 93.1115  | 30373444/32620495 |
| EFCalo | 91.8147  | 29950407/32620495 |
|  HLT   | 90.3955  | 29487453/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.39s/it]


HLT_e28_lhtight_nod0_ringer_v17_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.1980  | 21961165/23313826 |
|   L2   | 94.0757  | 21932642/23313826 |
| EFCalo | 93.2847  | 21748227/23313826 |
|  HLT   | 87.5873  | 20419953/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.20s/it]


HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.3658  | 1345279/1381675 |
|   L2   | 97.2001  | 1342989/1381675 |
| EFCalo | 95.1384  | 1314503/1381675 |
|  HLT   | 92.1650  | 1273421/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.07it/s]

HLT_e140_lhloose_nod0_ringer_v17
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.0559  | 59215/60389  |
|   L2   | 97.4383  | 58842/60389  |
| EFCalo | 96.3735  | 58199/60389  |
|  HLT   | 94.6133  | 57136/60389  |
+--------+----------+--------------+


In [22]:
eff_data18_Zee.save()

In [23]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v17.root')

2022-03-28 19:36:25,407 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v17.root


In [24]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v17_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v17'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.52s/it]


HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  2.6081  |  220827/8467098 |
|   L2   |  0.9533  |  80714/8467098  |
| EFCalo |  0.9195  |  77853/8467098  |
|  HLT   |  0.4571  |  38702/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.17s/it]


HLT_e28_lhtight_nod0_ringer_v17_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  1.1768  |  89777/7628804  |
|   L2   |  0.6828  |  52093/7628804  |
| EFCalo |  0.6517  |  49720/7628804  |
|  HLT   |  0.0441  |   3366/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.71s/it]


HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  1.1612  |  76195/6561985  |
|   L2   |  0.5974  |  39199/6561985  |
| EFCalo |  0.5639  |  37000/6561985  |
|  HLT   |  0.0523  |   3434/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.05s/it]

HLT_e140_lhloose_nod0_ringer_v17
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  2.4843  |  118662/4776518 |
|   L2   |  0.7698  |  36770/4776518  |
| EFCalo |  0.7319  |  34958/4776518  |
|  HLT   |  0.2798  |  13366/4776518  |
+--------+----------+-----------------+


In [25]:
eff_data18_JF17.save()